# Urban Heat Island (UHI) Benchmark Notebook 

## Challenge Overview

<p align="justify">Welcome to the EY Open Science AI & Data Challenge 2025! The objective of this challenge is to build a machine learning model to predict urban heat island (UHI) hotspots in a city. By the end of the challenge, you will have developed a regression model capable of predicting the intensity of the UHI effect.

Participants will be given ground-level air temperature data in an index format, which was collected on 24th July 2021 on traverse points in the Bronx and Manhattan regions of New York city. This dataset constitutes traverse points (latitude and longitude) and their corresponding UHI (Urban Heat Island) index values. Participants will use this dataset to build a regression model to predict UHI index values for a given set of locations. It is important to understand that the UHI Index at any given location is indicative of the relative temperature elevation at that specific point compared to the city's average temperature.

This challenge is designed for participants with varying skill levels in data science and programming, offering a great opportunity to apply your knowledge and enhance your capabilities in the field.</p>

<b>Challenge Aim: </b><p align="justify"> <p>

<p align="justify">In this notebook, we will demonstrate a basic model workflow that can serve as a starting point for the challenge. The basic model has been constructed to predict the Urban Heat Island (UHI) index using features from the Sentinel-2 satellite dataset as predictor variables. In this demonstration, we utilized three features from the Sentinel-2 dataset: band B01 (Coastal Aerosol), band B06 (Red Edge), and NDVI (Normalized Difference Vegetation Index) derived from bands B04 (Red) and B08 (Near Infrared). A random forest regression model was then trained using these features.
    
These features were extracted from a GeoTIFF image created by the Sentinel-2 sample notebook. For the sample model shown in this notebook, data from a single day (24th July 2021) was considered, assuming that the values of bands B01, B04, B06, and B08 for this specific date are representative of the UHI index behavior at any location. Participants should review the details of the Sentinel-2 sample notebook to gain an understanding of the data and options for modifying the output product. 
    
</p>

<p align="justify">Most of the functions presented in this notebook were adapted from the <a href="https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Example-Notebook">Sentinel-2-Level-2A notebook</a> found in the Planetary Computer portal.</p>

<p align="justify">Please note that this notebook is just a starting point. We have made many assumptions in this notebook that you may think are not best for solving the challenge effectively. You are encouraged to modify these functions, rewrite them, or try an entirely new approach.</p>


## Load In Dependencies

To run this demonstration notebook, you will need to have the following packages imported below installed. This may take some time.  

In [2]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
import geopandas as gpd

# Geospatial operations
import rasterio
from rasterio import windows  
from rasterio import features  
from rasterio import warp
from rasterio.warp import transform_bounds 
from rasterio.windows import from_bounds 

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from boruta import BorutaPy

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE

# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm

## Response Variable

Before building the model, we need to load in the Urban Heat Island (UHI) index training dataset. We have curated data for the New York region. The dataset consists of geo-locations (Longitude and Latitude), with additional fields including date & time of data collection and the UHI index for each location. 

In [3]:
# Load the training data from csv file and display the first few rows to inspect the data
ground_df = pd.read_csv("Integrated_UHI_Dataset_200m.csv")
ground_df.head()

,Longitude,Latitude,datetime,UHI Index,geometry,index_right,building_d,impervious,avg_temp,avg_wind_speed,avg_solar_flux
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,POINT (1009393.6056238374 235526.82444258165),27760,0.00005,1.962197,27.2,2.6,621.0
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,POINT (1009388.0927123841 235504.3501299469),27760,0.00005,1.962197,27.2,2.6,621.0
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,POINT (1009380.2758449932 235480.05175128343),27760,0.00005,1.962197,27.2,2.6,621.0
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,POINT (1009372.9197252728 235454.54061417878),27760,0.00005,1.962197,27.2,2.6,621.0
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,POINT (1009368.7914612402 235431.4629437401),27760,0.00005,1.962197,27.2,2.6,621.0


## Predictor Variables

<p align="justify">Now that we have our UHI data, it is time to gather the predictor variables from the Sentinel-2 dataset. Participants should review the provided Sentinel-2 sample notebook as it was used to create a sample GeoTIFF for this models. For a more in-depth look regarding the Sentinel-2 dataset and how to query it, see the Microsoft Planetary Computer example <a href="https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Example-Notebook">Sentinel-2 supplementary notebook</a>. </p>

<p align="justify">Sentinel-2 optical data provides high-resolution imagery that is sensitive to land surface characteristics, which are crucial for understanding urban heat dynamics. Band values such as B01 (Coastal aerosol), B06 (Red Edge), and NDVI (Normalized Difference Vegetation Index) derived from B04 (Red) and B08 (Near Infrared) help us in estimating the UHI index. Hence, we are choosing B01, B06, and NDVI as predictor variables for this experiment.</p>

<ul> 
<li>B01 - Reflectance values from the Coastal aerosol band, which help in assessing aerosol presence and improving atmospheric correction.</li>

<li>B06 - Reflectance values from the Red Edge band, which provide useful information for detecting vegetation, water bodies, and urban surfaces.</li>

<li>NDVI - Derived from B04 (Red) and B08 (Near Infrared), NDVI is an important indicator for vegetation health and land cover.</li>
</ul>


<h4 style="color:rgb(255, 255, 0)"><strong>Tip 1</strong></h4>
<p align="justify">Participants might explore other combinations of bands from the Sentinel-2 and from other satellite datasets as well. For example, you can use mathematical combinations of bands to generate various indices </a> which can then be used as features in your model. These bands or indices may provide insights into surface characteristics, vegetation, or built-up areas that could influence UHI patterns.

### Analyze the Sentinel-2 Data

<p align="justify">To obtain the Sentinel-2 data, we created a GeoTIFF image for a specific date and area of interest, which in this case is the Bronx and Manhattan regions of New York. The GeoTIFF product allows us to extract the relevant band values. In this example, we extracted B01 (Coastal Aerosol), B04 (Red), B06 (Red Edge), and B08 (Near Infrared) values for a single day (24th July, 2021). Participants should review the provided Sentinel-2 sample notebook for common output images (RGB, NDVI) and methods to alter the output.</p>

### Methods of Extracting Band Values from Sentinel-2 Data

There are two common methods to extract band values from Sentinel-2 data:

<ul>
    <li><strong>Using API Calls:</strong> Retrieve band values directly from Sentinel-2 datasets via APIs, such as the <code>planetary_computer</code>.</li>
</ul>
<ul>
    <li><strong>Using GeoTIFF Images:</strong> Create and download a GeoTIFF image containing the desired bands and extract the band values locally. The GeoTIFF image can represent any desired time period (single date or time series mosaic) and include any number of spectral bands.</li>
</ul>

Participants can select any of these approaches as per their convenience. Since our dataset is large, the API method can be time-consuming and resource-intensive. Therefore, in this sample notebook, we have opted for the second method and extracted the values for bands B01, B04, B08, and B06. Please refer to the Sentinel-2 sample notebook for details about the creation of the GeoTIFF image. 

<h4 style="color:rgb(255, 255, 0)"><strong>Tip 2</strong></h4>
<p align="justify"> Rather than extracting the bands for a single day coincident with the ground-based data collection, participants might explore other options to improve data quality. For example, one could select a different single date with minimal or no cloud cover or generate a median mosaic using several scenes within a time series. See the Sentinel-2 sample notebook for examples.</p>

### Downloading GeoTIFF Image

For building a sample model in this demonstration notebook, we have downloaded a GeoTIFF file locally for a single day (24th July 2021). The file is named <b>S2_sample.tiff</b>. This GeoTIFF file contains values of four bands: Band B01, Band B04, Band B06 and Band B08. In the subsequent section, we will use this GeoTIFF file to extract the band values for the geo-locations given in the training dataset to create the features.

First, let’s visualize the bands of the downloaded GeoTIFF image.

In [ ]:

# Reads and plots 11 bands (B01, B02, B03, B04,B05, B06, B07, B08, B8A, B11, B12) from the GeoTIFF file.

# Open the GeoTIFF file
tiff_path = "S2_AllBands.tiff"

# Read the bands from the GeoTIFF file
with rasterio.open(tiff_path) as src1:
    band1 = src1.read(1)  # Band [B01]
    band2 = src1.read(2)  # Band [B02]
    band3 = src1.read(3)  # Band [B03]
    band4 = src1.read(4)  # Band [B04]
    band5 = src1.read(5)  # Band [B05]
    band6 = src1.read(6)  # Band [B06]
    band7 = src1.read(7)  # Band [B07]
    band8 = src1.read(8)  # Band [B08]
    band9 = src1.read(9)  # Band [B8A]
    band10 = src1.read(10)  # Band [B11]
    band11 = src1.read(11)  # Band [B12]


# "B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"

# Plot the bands in a 4x4 grid
fig, axes = plt.subplots(3, 4, figsize=(10, 10))

# Flatten the axes for easier indexing
axes = axes.flatten()

# Plot the first band (B01)
im1 = axes[0].imshow(band1, cmap='viridis')
axes[0].set_title('Band [B01]')
fig.colorbar(im1, ax=axes[0])

# Plot the second band (B02)
im2 = axes[1].imshow(band2, cmap='viridis')
axes[1].set_title('Band [B02]')
fig.colorbar(im2, ax=axes[1])

# Plot the third band (B03)
im3 = axes[2].imshow(band3, cmap='viridis')                 
axes[2].set_title('Band [B03]')
fig.colorbar(im3, ax=axes[2])

# Plot the fourth band (B04)
im4 = axes[3].imshow(band4, cmap='viridis')
axes[3].set_title('Band [B04]')
fig.colorbar(im4, ax=axes[3])

# Plot the fifth band (B05)
im5 = axes[4].imshow(band5, cmap='viridis')
axes[4].set_title('Band [B05]')
fig.colorbar(im5, ax=axes[4])

# Plot the sixth band (B06)
im6 = axes[5].imshow(band6, cmap='viridis')
axes[5].set_title('Band [B06]')
fig.colorbar(im6, ax=axes[5])

# Plot the seventh band (B07)
im7 = axes[6].imshow(band7, cmap='viridis')
axes[6].set_title('Band [B07]')
fig.colorbar(im7, ax=axes[6])

# Plot the eighth band (B08)
im8 = axes[7].imshow(band8, cmap='viridis')
axes[7].set_title('Band [B08]')
fig.colorbar(im8, ax=axes[7])

# Plot the ninth band (B8A)
im9 = axes[8].imshow(band9, cmap='viridis')
axes[8].set_title('Band [B8A]')
fig.colorbar(im9, ax=axes[8])

# Plot the tenth band (B11)
im10 = axes[9].imshow(band10, cmap='viridis')
axes[9].set_title('Band [B11]')
fig.colorbar(im10, ax=axes[9])

# Plot the eleventh band (B12)
im11 = axes[10].imshow(band11, cmap='viridis')
axes[10].set_title('Band [B12]')
fig.colorbar(im11, ax=axes[10])


plt.tight_layout()
plt.show()


'\n# Reads and plots 11 bands (B01, B02, B03, B04,B05, B06, B07, B08, B8A, B11, B12) from the GeoTIFF file.\n\n# Open the GeoTIFF file\ntiff_path = "S2_AllBands.tiff"\n\n# Read the bands from the GeoTIFF file\nwith rasterio.open(tiff_path) as src1:\n    band1 = src1.read(1)  # Band [B01]\n    band2 = src1.read(2)  # Band [B02]\n    band3 = src1.read(3)  # Band [B03]\n    band4 = src1.read(4)  # Band [B04]\n    band5 = src1.read(5)  # Band [B05]\n    band6 = src1.read(6)  # Band [B06]\n    band7 = src1.read(7)  # Band [B07]\n    band8 = src1.read(8)  # Band [B08]\n    band9 = src1.read(9)  # Band [B8A]\n    band10 = src1.read(10)  # Band [B11]\n    band11 = src1.read(11)  # Band [B12]\n\n\n# "B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"\n\n# Plot the bands in a 4x4 grid\nfig, axes = plt.subplots(3, 4, figsize=(10, 10))\n\n# Flatten the axes for easier indexing\naxes = axes.flatten()\n\n# Plot the first band (B01)\nim1 = axes[0].imshow(band1, cmap=\'viridis\

<h4 style="color:rgb(255, 255, 0)"><strong>Tip 3</strong></h4>

Instead of a single point data extraction, participants might explore the approach of creating a focal buffer around the locations (e.g., 50 m, 100 m, 150 m etc). For example, if the specified distance was 50 m and the specified band was “Band 2”, then the value of the output pixels from this analysis would reflect the average values in band 2 within 50 meters of the specific location. This approach might help reduction in error associated with spatial autocorrelation. In this demonstration notebook, we are extracting the band data for each of the locations without creating a buffer zone.




### Extracting Band Values from the GeoTIFF Image


In [ ]:
'''
# Extracts satellite band values from a GeoTIFF based on coordinates from a csv file and returns them in a DataFrame.

def map_satellite_data(tiff_path, csv_path):
    
    # Load the GeoTIFF data
    data = rxr.open_rasterio(tiff_path)
    tiff_crs = data.rio.crs

    # Read the Excel file using pandas
    df = pd.read_csv(csv_path)
    latitudes = df['Latitude'].values
    longitudes = df['Longitude'].values

    # 3. Convert lat/long to the GeoTIFF's CRS
    # Create a Proj object for EPSG:4326 (WGS84 - lat/long) and the GeoTIFF's CRS
    proj_wgs84 = Proj(init='epsg:4326')  # EPSG:4326 is the common lat/long CRS
    proj_tiff = Proj(tiff_crs)
    
    # Create a transformer object
    transformer = Transformer.from_proj(proj_wgs84, proj_tiff)

    B01_values = []
    B02_values = []
    B03_values = []
    B04_values = []
    B05_values = []
    B06_values = []
    B07_values = []
    B08_values = []
    B8A_values = []
    B11_values = []
    B12_values = []

# Iterate over the latitudes and longitudes, and extract the corresponding band values
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping values"):
    # Assuming the correct dimensions are 'y' and 'x' (replace these with actual names from data.coords)
    
        B01_value = data.sel(x=lon, y=lat,  band=1, method="nearest").values
        B01_values.append(B01_value)
    
        B02_value = data.sel(x=lon, y=lat,  band=2, method="nearest").values
        B02_values.append(B02_value)

        B03_value = data.sel(x=lon, y=lat, band=3, method="nearest").values
        B03_values.append(B03_value)

        B04_value = data.sel(x=lon, y=lat, band=4, method="nearest").values
        B04_values.append(B04_value)

        B05_value = data.sel(x=lon, y=lat, band=5, method="nearest").values
        B05_values.append(B05_value)
        
        B06_value = data.sel(x=lon, y=lat, band=6, method="nearest").values
        B06_values.append(B06_value)

        B07_value = data.sel(x=lon, y=lat, band=7, method="nearest").values
        B07_values.append(B07_value)
    
        B08_value = data.sel(x=lon, y=lat, band=8, method="nearest").values
        B08_values.append(B08_value)

        B8A_value = data.sel(x=lon, y=lat, band=9, method="nearest").values
        B8A_values.append(B8A_value)

        B11_value = data.sel(x=lon, y=lat, band=10, method="nearest").values
        B11_values.append(B11_value)

        B12_value = data.sel(x=lon, y=lat, band=11, method="nearest").values
        B12_values.append(B12_value)

    # Create a DataFrame with the band values
    # Create a DataFrame to store the band values
    df = pd.DataFrame()
    df['B01'] = B01_values
    df['B02'] = B02_values
    df['B03'] = B03_values
    df['B04'] = B04_values
    df['B05'] = B05_values
    df['B06'] = B06_values
    df['B07'] = B07_values
    df['B08'] = B08_values
    df['B8A'] = B8A_values
    df['B11'] = B11_values
    df['B12'] = B12_values
    
    return df
'''

'\n# Extracts satellite band values from a GeoTIFF based on coordinates from a csv file and returns them in a DataFrame.\n\ndef map_satellite_data(tiff_path, csv_path):\n\n    # Load the GeoTIFF data\n    data = rxr.open_rasterio(tiff_path)\n    tiff_crs = data.rio.crs\n\n    # Read the Excel file using pandas\n    df = pd.read_csv(csv_path)\n    latitudes = df[\'Latitude\'].values\n    longitudes = df[\'Longitude\'].values\n\n    # 3. Convert lat/long to the GeoTIFF\'s CRS\n    # Create a Proj object for EPSG:4326 (WGS84 - lat/long) and the GeoTIFF\'s CRS\n    proj_wgs84 = Proj(init=\'epsg:4326\')  # EPSG:4326 is the common lat/long CRS\n    proj_tiff = Proj(tiff_crs)\n\n    # Create a transformer object\n    transformer = Transformer.from_proj(proj_wgs84, proj_tiff)\n\n    B01_values = []\n    B02_values = []\n    B03_values = []\n    B04_values = []\n    B05_values = []\n    B06_values = []\n    B07_values = []\n    B08_values = []\n    B8A_values = []\n    B11_values = []\n    B12_

In [4]:
def map_satellite_data_buffer(tiff_path, csv_path, buffer_m=50):
    """
    Maps satellite band values by creating a buffer zone (in meters) around each coordinate.
    For each location, it computes the average value within the buffered window.
    
    Parameters:
      tiff_path: path to the GeoTIFF file.
      csv_path: path to the CSV file with 'Latitude' and 'Longitude' columns.
      buffer_m: buffer radius in meters (e.g., 50, 100, 150, 200).
      
    Returns:
      df: DataFrame with averaged band values from the buffer zone.
    """
    # Load the GeoTIFF data
    data = rxr.open_rasterio(tiff_path)
    tiff_crs = data.rio.crs

    # Read the CSV file with coordinates
    df_coords = pd.read_csv(csv_path)
    latitudes = df_coords['Latitude'].values
    longitudes = df_coords['Longitude'].values

    # Create Proj objects for coordinate transformation
    proj_wgs84 = Proj("epsg:4326")  # WGS84 lat/long
    proj_tiff = Proj(tiff_crs)
    transformer = Transformer.from_proj(proj_wgs84, proj_tiff)

    # Determine the buffer value in raster units.
    # If the GeoTIFF is in EPSG:4326, convert buffer from meters to degrees (approximation)
    if tiff_crs.to_string().lower().startswith("epsg:4326"):
        buffer_val = buffer_m / 111320.0  # approximate conversion: 1 deg ~111.32 km
    else:
        buffer_val = buffer_m  # assume CRS units are in meters

    # Lists to hold output values for each band
    B01_values = []
    B02_values = []
    B03_values = []
    B04_values = []
    B05_values = []
    B06_values = []
    B07_values = []
    B08_values = []
    B8A_values = []
    B11_values = []
    B12_values = []

    # Iterate over coordinates
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc=f"Mapping values with {buffer_m} m buffer"):
        # Determine the window (bounds) in the coordinate system of the raster.
        if tiff_crs.to_string().lower().startswith("epsg:4326"):
            left = lon - buffer_val
            right = lon + buffer_val
            bottom = lat - buffer_val
            top = lat + buffer_val
        else:
            # Transform the coordinate from lat/long to the raster CRS
            x, y = transformer.transform(lat, lon)
            left = x - buffer_val
            right = x + buffer_val
            bottom = y - buffer_val
            top = y + buffer_val

        # Clip the data using the computed bounding box.
        # This produces a subset of the raster covering the buffer window.
        clipped = data.rio.clip_box(minx=left, miny=bottom, maxx=right, maxy=top)
        
        # Compute the average value within the window for each band.
        # (Assuming bands are arranged as: band=1:B01, band=2:B02, band=3:B03, band=4:B04, band=5:B05, band=6:B06, band=7:B07, band=8:B08, band=9:B8A, band=10:B11, band=11:B12)
        B01_val = np.nanmean(clipped.sel(band=1).values)
        B02_val = np.nanmean(clipped.sel(band=2).values)
        B03_val = np.nanmean(clipped.sel(band=3).values)
        B04_val = np.nanmean(clipped.sel(band=4).values)
        B05_val = np.nanmean(clipped.sel(band=5).values)
        B06_val = np.nanmean(clipped.sel(band=6).values)
        B07_val = np.nanmean(clipped.sel(band=7).values)
        B08_val = np.nanmean(clipped.sel(band=8).values)
        B8A_val = np.nanmean(clipped.sel(band=9).values)
        B11_val = np.nanmean(clipped.sel(band=10).values)
        B12_val = np.nanmean(clipped.sel(band=11).values)

        B01_values.append(B01_val)
        B02_values.append(B02_val)
        B03_values.append(B03_val)
        B04_values.append(B04_val)
        B05_values.append(B05_val)
        B06_values.append(B06_val)
        B07_values.append(B07_val)
        B08_values.append(B08_val)
        B8A_values.append(B8A_val)
        B11_values.append(B11_val)
        B12_values.append(B12_val)

    # Create a DataFrame with the averaged band values.
    df = pd.DataFrame({
        'B01': B01_values,
        'B02': B02_values,
        'B03': B03_values,
        'B04': B04_values,
        'B05': B05_values,
        'B06': B06_values,
        'B07': B07_values,
        'B08': B08_values,
        'B8A': B8A_values,
        'B11': B11_values,
        'B12': B12_values
    })
    return df





In [ ]:
# Mapping satellite data with training data.
# final_data = map_satellite_data('S2_AllBands.tiff', 'Training_data_uhi_index.csv')
# Example usage:
#if __name__ == "__main__":
    # For example, setting buffer_m to 50 meters. You can change this value (50, 100, 150, 200)
# final_data_50m = map_satellite_data_buffer("S2_AllBands.tiff", "Training_data_uhi_index.csv", buffer_m=50)
    # print(final_data_50m.head())
    # You can save the output to a CSV if desired:
    # final_data_50m.to_csv("buffered_satellite_values_50m.csv", index=False)

    # For example, setting buffer_m to 100 meters. You can change this value (50, 100, 150, 200)
# final_data_100m = map_satellite_data_buffer("S2_AllBands.tiff", "Training_data_uhi_index.csv", buffer_m=100)
    # print(final_data_100m.head())
    # You can save the output to a CSV if desired:
    # final_data_100m.to_csv("buffered_satellite_values_100m.csv", index=False)

    # For example, setting buffer_m to 150 meters. You can change this value (50, 100, 150, 200)
# final_data_150m = map_satellite_data_buffer("S2_AllBands.tiff", "Training_data_uhi_index.csv", buffer_m=150)
    # print(final_data_150m.head())
    # You can save the output to a CSV if desired:
    # final_data_150m.to_csv("buffered_satellite_values_150m.csv", index=False)

    # For example, setting buffer_m to 200 meters. You can change this value (50, 100, 150, 200)
final_data_200m = map_satellite_data_buffer("S2_AllBands.tiff", "Integrated_UHI_Dataset_200m.csv", buffer_m=200)
final_data_200m.head()
    # You can save the output to a CSV if desired:
final_data_200m.to_csv("buffered_satellite_values_200m.csv", index=False)

Mapping values with 200 m buffer: 100%|██████████| 11229/11229 [16:58<00:00, 11.02it/s]


In [ ]:
# final_data.head()

,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12
0,846.0,1036.0,1502.0,1906.0,1906.0,1906.0,1906.0,1906.0,1906.0,1906.0,1906.0
1,846.0,1036.0,1502.0,1906.0,1906.0,1906.0,1906.0,1906.0,1906.0,1906.0,1906.0
2,846.0,709.0,1668.0,2190.0,2190.0,2190.0,2190.0,2190.0,2190.0,2190.0,2190.0
3,846.0,657.0,1668.0,2182.0,2182.0,2182.0,2182.0,2182.0,2182.0,2182.0,2182.0
4,846.0,745.0,1728.0,2112.0,2112.0,2112.0,2112.0,2112.0,2112.0,2112.0,2112.0


In [ ]:
# Calculate NDVI, NDBI, NDWI, NDMI (Normalized Difference Vegetation Index) and handle division by zero by replacing infinities with NaN.
# See the Sentinel-2 sample notebook for more information about the NDVI, NDBI, NDWI and NDMI indices
# final_data['NDVI'] = (final_data['B08'] - final_data['B04']) / (final_data['B08'] + final_data['B04'])
# final_data['NDVI'] = final_data['NDVI'].replace([np.inf, -np.inf], np.nan)

# final_data['NDBI'] = (final_data['B11'] - final_data['B08']) / (final_data['B11'] + final_data['B08'])
# final_data['NDBI'] = final_data['NDBI'].replace([np.inf, -np.inf], np.nan)

# final_data['NDWI'] = (final_data['B03'] - final_data['B08']) / (final_data['B03'] + final_data['B08'])
# final_data['NDWI'] = final_data['NDWI'].replace([np.inf, -np.inf], np.nan)

# final_data['NDMI'] = (final_data['B08'] - final_data['B11']) / (final_data['B08'] + final_data['B11'])
# final_data['NDMI'] = final_data['NDMI'].replace([np.inf, -np.inf], np.nan)

#    df_bands['NDBI'] = (df_bands['B11'] - df_bands['B08']) / (df_bands['B11'] + df_bands['B08'] + 1e-6)
#    df_bands['NDWI'] = (df_bands['B03'] - df_bands['B08']) / (df_bands['B03'] + df_bands['B08'] + 1e-6)
#    df_bands['NDMI'] = (df_bands['B08'] - df_bands['B11']) / (df_bands['B08'] + df_bands['B11'] + 1e-6)
#    df_bands['NDVI'] = (df_bands['B08'] - df_bands['B04']) / (df_bands['B08'] + df_bands['B04'] + 1e-6)




In [ ]:
# Calculate NDVI, NDBI, NDWI, NDMI (Normalized Difference Vegetation Index) and handle division by zero by replacing infinities with NaN.
# See the Sentinel-2 sample notebook for more information about the NDVI, NDBI, NDWI and NDMI indices
# final_data_50m['NDVI'] = (final_data_50m['B08'] - final_data_50m['B04']) / (final_data_50m['B08'] + final_data_50m['B04'])
# final_data_50m['NDVI'] = final_data_50m['NDVI'].replace([np.inf, -np.inf], np.nan)

# final_data_50m['NDBI'] = (final_data_50m['B11'] - final_data_50m['B08']) / (final_data_50m['B11'] + final_data_50m['B08'])
# final_data_50m['NDBI'] = final_data_50m['NDBI'].replace([np.inf, -np.inf], np.nan)

# final_data_50m['NDWI'] = (final_data_50m['B03'] - final_data_50m['B08']) / (final_data_50m['B03'] + final_data_50m['B08'])
# final_data_50m['NDWI'] = final_data_50m['NDWI'].replace([np.inf, -np.inf], np.nan)

# final_data_50m['NDMI'] = (final_data_50m['B08'] - final_data_50m['B11']) / (final_data_50m['B08'] + final_data_50m['B11'])
# final_data_50m['NDMI'] = final_data_50m['NDMI'].replace([np.inf, -np.inf], np.nan)

#    df_bands['NDBI'] = (df_bands['B11'] - df_bands['B08']) / (df_bands['B11'] + df_bands['B08'] + 1e-6)
#    df_bands['NDWI'] = (df_bands['B03'] - df_bands['B08']) / (df_bands['B03'] + df_bands['B08'] + 1e-6)
#    df_bands['NDMI'] = (df_bands['B08'] - df_bands['B11']) / (df_bands['B08'] + df_bands['B11'] + 1e-6)
#    df_bands['NDVI'] = (df_bands['B08'] - df_bands['B04']) / (df_bands['B08'] + df_bands['B04'] + 1e-6)



In [ ]:
# Calculate NDVI, NDBI, NDWI, NDMI (Normalized Difference Vegetation Index) and handle division by zero by replacing infinities with NaN.
# See the Sentinel-2 sample notebook for more information about the NDVI, NDBI, NDWI and NDMI indices
# final_data_100m['NDVI'] = (final_data_100m['B08'] - final_data_100m['B04']) / (final_data_100m['B08'] + final_data_100m['B04'])
# final_data_100m['NDVI'] = final_data_100m['NDVI'].replace([np.inf, -np.inf], np.nan)

# final_data_100m['NDBI'] = (final_data_100m['B11'] - final_data_100m['B08']) / (final_data_100m['B11'] + final_data_100m['B08'])
# final_data_100m['NDBI'] = final_data_100m['NDBI'].replace([np.inf, -np.inf], np.nan)

# final_data_100m['NDWI'] = (final_data_100m['B03'] - final_data_100m['B08']) / (final_data_100m['B03'] + final_data_100m['B08'])
# final_data_100m['NDWI'] = final_data_100m['NDWI'].replace([np.inf, -np.inf], np.nan)

# final_data_100m['NDMI'] = (final_data_100m['B08'] - final_data_100m['B11']) / (final_data_100m['B08'] + final_data_100m['B11'])
# final_data_100m['NDMI'] = final_data_100m['NDMI'].replace([np.inf, -np.inf], np.nan)

#    df_bands['NDBI'] = (df_bands['B11'] - df_bands['B08']) / (df_bands['B11'] + df_bands['B08'] + 1e-6)
#    df_bands['NDWI'] = (df_bands['B03'] - df_bands['B08']) / (df_bands['B03'] + df_bands['B08'] + 1e-6)
#    df_bands['NDMI'] = (df_bands['B08'] - df_bands['B11']) / (df_bands['B08'] + df_bands['B11'] + 1e-6)
#    df_bands['NDVI'] = (df_bands['B08'] - df_bands['B04']) / (df_bands['B08'] + df_bands['B04'] + 1e-6)


In [ ]:
# Calculate NDVI, NDBI, NDWI, NDMI (Normalized Difference Vegetation Index) and handle division by zero by replacing infinities with NaN.
# See the Sentinel-2 sample notebook for more information about the NDVI, NDBI, NDWI and NDMI indices
# final_data_150m['NDVI'] = (final_data_150m['B08'] - final_data_150m['B04']) / (final_data_150m['B08'] + final_data_150m['B04'])
# final_data_150m['NDVI'] = final_data_150m['NDVI'].replace([np.inf, -np.inf], np.nan)

# final_data_150m['NDBI'] = (final_data_150m['B11'] - final_data_150m['B08']) / (final_data_150m['B11'] + final_data_150m['B08'])
# final_data_150m['NDBI'] = final_data_150m['NDBI'].replace([np.inf, -np.inf], np.nan)

# final_data_150m['NDWI'] = (final_data_150m['B03'] - final_data_150m['B08']) / (final_data_150m['B03'] + final_data_150m['B08'])
# final_data_150m['NDWI'] = final_data_150m['NDWI'].replace([np.inf, -np.inf], np.nan)

# final_data_150m['NDMI'] = (final_data_150m['B08'] - final_data_150m['B11']) / (final_data_150m['B08'] + final_data_150m['B11'])
# final_data_150m['NDMI'] = final_data_150m['NDMI'].replace([np.inf, -np.inf], np.nan)

#    df_bands['NDBI'] = (df_bands['B11'] - df_bands['B08']) / (df_bands['B11'] + df_bands['B08'] + 1e-6)
#    df_bands['NDWI'] = (df_bands['B03'] - df_bands['B08']) / (df_bands['B03'] + df_bands['B08'] + 1e-6)
#    df_bands['NDMI'] = (df_bands['B08'] - df_bands['B11']) / (df_bands['B08'] + df_bands['B11'] + 1e-6)
#    df_bands['NDVI'] = (df_bands['B08'] - df_bands['B04']) / (df_bands['B08'] + df_bands['B04'] + 1e-6)


In [6]:
# Calculate NDVI, NDBI, NDWI, NDMI (Normalized Difference Vegetation Index) and handle division by zero by replacing infinities with NaN.
# See the Sentinel-2 sample notebook for more information about the NDVI, NDBI, NDWI and NDMI indices
final_data_200m['NDVI'] = (final_data_200m['B08'] - final_data_200m['B04']) / (final_data_200m['B08'] + final_data_200m['B04'])
final_data_200m['NDVI'] = final_data_200m['NDVI'].replace([np.inf, -np.inf], np.nan)

final_data_200m['NDBI'] = (final_data_200m['B11'] - final_data_200m['B08']) / (final_data_200m['B11'] + final_data_200m['B08'])
final_data_200m['NDBI'] = final_data_200m['NDBI'].replace([np.inf, -np.inf], np.nan)

final_data_200m['NDWI'] = (final_data_200m['B03'] - final_data_200m['B08']) / (final_data_200m['B03'] + final_data_200m['B08'])
final_data_200m['NDWI'] = final_data_200m['NDWI'].replace([np.inf, -np.inf], np.nan)

final_data_200m['NDMI'] = (final_data_200m['B08'] - final_data_200m['B11']) / (final_data_200m['B08'] + final_data_200m['B11'])
final_data_200m['NDMI'] = final_data_200m['NDMI'].replace([np.inf, -np.inf], np.nan)

#    df_bands['NDBI'] = (df_bands['B11'] - df_bands['B08']) / (df_bands['B11'] + df_bands['B08'] + 1e-6)
#    df_bands['NDWI'] = (df_bands['B03'] - df_bands['B08']) / (df_bands['B03'] + df_bands['B08'] + 1e-6)
#    df_bands['NDMI'] = (df_bands['B08'] - df_bands['B11']) / (df_bands['B08'] + df_bands['B11'] + 1e-6)
#    df_bands['NDVI'] = (df_bands['B08'] - df_bands['B04']) / (df_bands['B08'] + df_bands['B04'] + 1e-6)


In [7]:
final_data_200m.head()

,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,NDMI
0,1049.372992,1176.008923,1337.114813,1391.926829,1646.461035,2169.419988,2366.522308,2382.242118,2456.720999,2307.096966,1957.656752,0.262393,-0.016025,-0.280997,0.016025
1,1049.372992,1176.008923,1337.114813,1391.926829,1646.461035,2169.419988,2366.522308,2382.242118,2456.720999,2307.096966,1957.656752,0.262393,-0.016025,-0.280997,0.016025
2,1032.155265,1166.573468,1328.574063,1380.391434,1643.679358,2183.863772,2388.671624,2407.468769,2484.348602,2315.012493,1955.017252,0.271150,-0.019578,-0.288780,0.019578
3,1035.991672,1167.440809,1330.071386,1380.349197,1644.202855,2181.948840,2386.545509,2408.089827,2485.607971,2321.805473,1962.969661,0.271283,-0.018242,-0.288382,0.018242
4,1036.517422,1170.410569,1333.412311,1383.382695,1647.285134,2182.242741,2386.358304,2407.404762,2485.654472,2327.103949,1969.113821,0.270134,-0.016961,-0.287101,0.016961


## Joining the predictor variables and response variables
Now that we have extracted our predictor variables, we need to join them onto the response variable . We use the function <i><b>combine_two_datasets</b></i> to combine the predictor variables and response variables.The <i><b>concat</b></i> function from pandas comes in handy here.

In [8]:
# Combine two datasets vertically (along columns) using pandas concat function.
def combine_two_datasets(dataset1,dataset2):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [9]:
# Combining ground data and final data into a single dataset.
# final_data_50m, final_data_100m, final_data_150m, final_data_200m
uhi_data = combine_two_datasets(ground_df,final_data_200m)
uhi_data.head()

,Longitude,Latitude,datetime,UHI Index,geometry,index_right,building_d,impervious,avg_temp,avg_wind_speed,...,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,NDMI
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,POINT (1009393.6056238374 235526.82444258165),27760,0.00005,1.962197,27.2,2.6,...,2169.419988,2366.522308,2382.242118,2456.720999,2307.096966,1957.656752,0.262393,-0.016025,-0.280997,0.016025
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,POINT (1009388.0927123841 235504.3501299469),27760,0.00005,1.962197,27.2,2.6,...,2169.419988,2366.522308,2382.242118,2456.720999,2307.096966,1957.656752,0.262393,-0.016025,-0.280997,0.016025
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,POINT (1009380.2758449932 235480.05175128343),27760,0.00005,1.962197,27.2,2.6,...,2183.863772,2388.671624,2407.468769,2484.348602,2315.012493,1955.017252,0.271150,-0.019578,-0.288780,0.019578
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,POINT (1009372.9197252728 235454.54061417878),27760,0.00005,1.962197,27.2,2.6,...,2181.948840,2386.545509,2408.089827,2485.607971,2321.805473,1962.969661,0.271283,-0.018242,-0.288382,0.018242
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,POINT (1009368.7914612402 235431.4629437401),27760,0.00005,1.962197,27.2,2.6,...,2182.242741,2386.358304,2407.404762,2485.654472,2327.103949,1969.113821,0.270134,-0.016961,-0.287101,0.016961


## Removing duplicates
Identical or duplicate entries are removed based on specific columns, in our case [ 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12','NDVI', 'NDBI', 'NDWI', 'NDMI']. This ensures that the dataset is not biased or skewed by repetitive data, allowing the model to train on unique, relevant observations.

In [10]:
# Remove duplicate rows from the DataFrame based on specified columns and keep the first occurrence
columns_to_check = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12','NDVI', 'NDBI', 'NDWI', 'NDMI', 'building_d','impervious','avg_temp','avg_wind_speed','avg_solar_flux']
for col in columns_to_check:
    # Check if the value is a numpy array and has more than one dimension
    uhi_data[col] = uhi_data[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)

# Now remove duplicates
uhi_data = uhi_data.drop_duplicates(subset=columns_to_check, keep='first')
uhi_data.head()


,Longitude,Latitude,datetime,UHI Index,geometry,index_right,building_d,impervious,avg_temp,avg_wind_speed,...,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,NDMI
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,POINT (1009393.6056238374 235526.82444258165),27760,0.00005,1.962197,27.2,2.6,...,2169.419988,2366.522308,2382.242118,2456.720999,2307.096966,1957.656752,0.262393,-0.016025,-0.280997,0.016025
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,POINT (1009380.2758449932 235480.05175128343),27760,0.00005,1.962197,27.2,2.6,...,2183.863772,2388.671624,2407.468769,2484.348602,2315.012493,1955.017252,0.271150,-0.019578,-0.288780,0.019578
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,POINT (1009372.9197252728 235454.54061417878),27760,0.00005,1.962197,27.2,2.6,...,2181.948840,2386.545509,2408.089827,2485.607971,2321.805473,1962.969661,0.271283,-0.018242,-0.288382,0.018242
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,POINT (1009368.7914612402 235431.4629437401),27760,0.00005,1.962197,27.2,2.6,...,2182.242741,2386.358304,2407.404762,2485.654472,2327.103949,1969.113821,0.270134,-0.016961,-0.287101,0.016961
5,-73.909280,40.812777,2021-07-24 15:53:00,1.021634,POINT (1009362.3592352993 235406.56120849005),27760,0.00005,1.962197,27.2,2.6,...,2181.183819,2385.671624,2405.588935,2486.767995,2328.472338,1969.781678,0.271631,-0.016290,-0.288321,0.016290


In [11]:
# Resetting the index of the dataset
uhi_data=uhi_data.reset_index(drop=True)

## Model Building

<p align="justify"> Now let us select the columns required for our model building exercise. We will consider only Band B01, Band B06 and NDVI from the Sentinel-2 data as our predictor variables. It does not make sense to use latitude and longitude as predictor variables, as they do not have any direct impact on predicting the UHI index.</p>


In [12]:
# Retaining only the columns for B01,B02,B05,B06,B07,B08A,B12,NDVI,NDBI,NDMI,NDWI, building_d,impervious,avg_temp,avg_wind_speed,avg_solar_flux and UHI Index in the dataset. 
uhi_data = uhi_data[['B01','B02','B05','B06','B07','B8A','B12','NDVI','NDBI','NDMI','NDWI','UHI Index', 'building_d', 'impervious', 'avg_temp', 'avg_wind_speed', 'avg_solar_flux']]

In [ ]:
    # You can save the output to a CSV if desired:
uhi_data.to_csv("model_building_200m.csv", index=False)

### Train and Test Split 

<p align="justify">We will now split the data into 70% training data and 30% test data. Scikit-learn alias “sklearn” is a robust library for machine learning in Python. The scikit-learn library has a <i><b>model_selection</b></i> module in which there is a splitting function <i><b>train_test_split</b></i>. You can use the same.</p>

In [53]:
# Split the data into features (X) and target (y), and then into training and testing sets
X = uhi_data.drop(columns=['UHI Index']).values
y = uhi_data ['UHI Index'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=123)

### Feature Scaling 

<p align="justify"> Before initiating the model training we may have to execute different data pre-processing steps. Here we are demonstrating the scaling of B01, B06, NDVI variable by using Standard Scaler.</p>

<p align = "justify">Feature Scaling is a data preprocessing step for numerical features. Many machine learning algorithms like Gradient descent methods, KNN algorithm, linear and logistic regression, etc. require data scaling to produce good results. Scikit learn provides functions that can be used to apply data scaling. Here we are using Standard Scaler. The idea behind Standard Scaler is that it will transform your data such that its distribution will have a mean value 0 and standard deviation of 1.</p>

<h4 style="color:rgb(255, 255, 0)"><strong>Tip 4</strong></h4>
<p align="justify">There are many data preprocessing methods available, which might help to improve the model performance. Participants should explore various suitable preprocessing methods as well as different machine learning algorithms to build a robust model.</p>

In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assume df is your complete dataset with the following predictor features and target:
all_features = ['B01','B02','B05','B06','B07','B8A','B12',
                 'NDVI','NDBI','NDMI','NDWI',
                 'building_d','impervious','avg_temp','avg_wind_speed','avg_solar_flux']
target = 'UHI Index'

# Split predictors and target
X = uhi_data[all_features]
y = uhi_data[target]

#########################################
# 1. Correlation Filtering
#########################################
# Compute absolute correlation matrix
corr_matrix = X.corr().abs()
# Get upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# Find features with correlation greater than threshold (e.g., 0.90)
to_drop = [col for col in upper.columns if any(upper[col] > 0.90)]
print("Features to drop due to high correlation:", to_drop)
# Drop highly correlated features
X_corr_filtered = X.drop(columns=to_drop)
print("Remaining features after correlation filtering:", X_corr_filtered.columns.tolist())


Features to drop due to high correlation: ['B02', 'B05', 'B07', 'B8A', 'B12', 'NDBI', 'NDMI', 'NDWI']
Remaining features after correlation filtering: ['B01', 'B06', 'NDVI', 'building_d', 'impervious', 'avg_temp', 'avg_wind_speed', 'avg_solar_flux']


In [17]:

#########################################
# 2. Recursive Feature Elimination (RFE)
#########################################
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

# Set up a RandomForestRegressor with your tuned parameters
rf_estimator = RandomForestRegressor(n_estimators=500,
                                     bootstrap=False,
                                     max_depth=None,
                                     max_features=0.5,
                                     min_samples_leaf=1,
                                     min_samples_split=2,
                                     random_state=42)

# Use RFE to select, say, the top 10 features (adjust n_features_to_select as needed)
rfe_selector = RFE(estimator=rf_estimator, n_features_to_select=10)
rfe_selector.fit(X_corr_filtered, y)
selected_features_rfe = X_corr_filtered.columns[rfe_selector.support_].tolist()
print("Features selected by RFE:", selected_features_rfe)



Features selected by RFE: ['B01', 'B06', 'NDVI', 'building_d', 'impervious', 'avg_temp', 'avg_wind_speed', 'avg_solar_flux']


In [18]:
#########################################
# 3. Lasso Feature Selection
#########################################
from sklearn.linear_model import LassoCV

# Run LassoCV to determine important features (L1 regularization shrinks unimportant coefficients to zero)
lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_corr_filtered, y)
coef_series = pd.Series(lasso.coef_, index=X_corr_filtered.columns)
print("Lasso coefficients:")
print(coef_series)
selected_features_lasso = coef_series[coef_series != 0].index.tolist()
print("Features selected by Lasso:", selected_features_lasso)



Lasso coefficients:
B01               0.000024
B06               0.000006
NDVI             -0.000000
building_d        0.000000
impervious       -0.000000
avg_temp          0.000000
avg_wind_speed    0.000000
avg_solar_flux   -0.000000
dtype: float64
Features selected by Lasso: ['B01', 'B06']


In [19]:
#########################################
# 4. Boruta Feature Selection
#########################################
# Boruta is a wrapper method built on top of tree-based models.
# You might need to install BorutaPy: pip install Boruta
from boruta import BorutaPy

rf_for_boruta = RandomForestRegressor(n_estimators=500, random_state=42)
boruta_selector = BorutaPy(rf_for_boruta, n_estimators='auto', random_state=42)
boruta_selector.fit(X_corr_filtered.values, y.values)
selected_features_boruta = X_corr_filtered.columns[boruta_selector.support_].tolist()
print("Features selected by Boruta:", selected_features_boruta)

#########################################
# 5. Buffering Methods (Optional - Geospatial Data)
#########################################
# If your dataset contains spatial coordinates (e.g. Latitude and Longitude)
# and you suspect local noise, you can aggregate measurements based on a spatial buffer.
# For example, using geopandas:
#
# import geopandas as gpd
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs='EPSG:4326')
# # Convert to a metric CRS for buffering (e.g., EPSG:3857)
# gdf = gdf.to_crs(epsg=3857)
# # Create buffers around each point (e.g., 100m)
# gdf['buffer'] = gdf.geometry.buffer(100)
# # Spatial join or aggregate (mean/median) features within each buffer here.
# # This step can reduce noise by averaging nearby measurements.
#
# (Implementing buffering requires domain knowledge on appropriate buffer sizes
# and a spatial aggregation strategy.)

#########################################
# How to Proceed:
# - Compare the feature sets selected by these methods.
# - You can then retrain your model on one of the reduced feature sets.
# - For example, re-run your RandomForestRegressor training with X_corr_filtered[selected_features_boruta]:
#
# X_selected = X_corr_filtered[selected_features_boruta]
# Then split X_selected and y into train/test sets, scale if needed,
# and train your model.
#########################################

Features selected by Boruta: ['B01', 'B06', 'NDVI', 'building_d', 'impervious', 'avg_temp', 'avg_wind_speed', 'avg_solar_flux']


In [21]:
# Split the data into features (X) and target (y), and then into training and testing sets
X_selected = X_corr_filtered[selected_features_boruta]
# X = uhi_data.drop(columns=['UHI Index']).values
y = uhi_data ['UHI Index'].values
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3,random_state=123)

In [54]:
# Scale the training and test data using standardscaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Alternative Options for Scaling Data:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, Normalizer

# Example data split (assuming X_train and X_test are defined)
# Standard Scaler (baseline)
sc_standard = StandardScaler()
X_train_standard = sc_standard.fit_transform(X_train)
X_test_standard = sc_standard.transform(X_test)

# MinMax Scaler
sc_minmax = MinMaxScaler()
X_train_minmax = sc_minmax.fit_transform(X_train)
X_test_minmax = sc_minmax.transform(X_test)

# Robust Scaler
sc_robust = RobustScaler()
X_train_robust = sc_robust.fit_transform(X_train)
X_test_robust = sc_robust.transform(X_test)

# MaxAbs Scaler
sc_maxabs = MaxAbsScaler()
X_train_maxabs = sc_maxabs.fit_transform(X_train)
X_test_maxabs = sc_maxabs.transform(X_test)

# Normalizer (applies row-wise normalization)
sc_normalizer = Normalizer()
X_train_normalized = sc_normalizer.fit_transform(X_train)
X_test_normalized = sc_normalizer.transform(X_test)

In [ ]:

# Apply PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  # Keep components explaining 95% of variance

X_train_pca_standard = pca.fit_transform(X_train_standard)
X_test_pca_standard = pca.transform(X_test_standard)

X_train_pca_minmax = pca.fit_transform(X_train_minmax)
X_test_pca_minmax = pca.transform(X_test_minmax)

X_train_pca_robust = pca.fit_transform(X_train_robust)
X_test_pca_robust = pca.transform(X_test_robust)

X_train_pca_normalized = pca.fit_transform(X_train_normalized)
X_test_pca_normalized = pca.transform(X_test_normalized)

# Explore PCA results (e.g., explained variance ratio)
print("Explained variance ratio (Standard Scaler):", pca.explained_variance_ratio_)
print("Number of components (Standard Scaler):", pca.n_components_)

Explained variance ratio (Standard Scaler): [0.32741578 0.21050175 0.11595001 0.10877599 0.08605663 0.08000288
 0.05417991]
Number of components (Standard Scaler): 7


### Model Training

<p align="justify">Now that we have the data in a format suitable for machine learning, we can begin training a model. In this demonstration notebook, we will use a random forest regression model from the scikit-learn library. This library offers a wide range of other models, each with extensive parameter tuning and customization capabilities.</p>

<p align="justify">Scikit-learn models require the separation of predictor variables and the response variable. We will store the predictor variables (Band B01, B06, NDVI) in array X and the response variable (UHI index) in array Y. It is important not to include the response variable in array X. Additionally, since latitude and longitude do not contribute to the prediction of UHI in this case, we will drop those as well.</p>


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Define a parameter grid for Random Forest hyperparameters.
param_grid = {
    'n_estimators': [200, 300, 500],        # Number of trees.
    'max_depth': [None, 5, 10, 20],           # Max depth of trees.
    'max_features': ['sqrt', 0.5, 1.0],       # Feature subset splitting (0.5 means 50% of features).
    'min_samples_split': [2, 5, 10],          # Minimum samples to split a node.
    'min_samples_leaf': [1, 5, 10],           # Minimum samples required at a leaf node.
    'bootstrap': [True, False]              # Whether bootstrap sampling is used.
}

# Initialize a RandomForestRegressor with a fixed random state.
rf = RandomForestRegressor(random_state=42)

# Set up the grid search with 5-fold cross-validation.
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=5,
                           scoring='r2',
                           n_jobs=-1,
                           verbose=1)

# Fit grid search to the training data.
grid_search.fit(X_train, y_train)

print("Best hyperparameters found: ", grid_search.best_params_)
print("Best cross-validation R²: ", grid_search.best_score_)

# Retrieve the best estimator.
best_rf_model = grid_search.best_estimator_

# Evaluate on the test set.
y_test_pred = best_rf_model.predict(X_test)
print("Test R²:", r2_score(y_test, y_test_pred))

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
Best hyperparameters found:  {'bootstrap': False, 'max_depth': None, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Best cross-validation R²:  0.880309203893629
Test R²: 0.9116995528631416

In [23]:
# Train the Random Forest model on the training data using Standard Scaler
# model = RandomForestRegressor(n_estimators=100, random_state=42)
model = RandomForestRegressor(bootstrap=False, max_depth=None, max_features=0.5, min_samples_leaf=1, min_samples_split=2, n_estimators=500, random_state=42)
model.fit(X_train,y_train)



RandomForestRegressor(bootstrap=False, max_features=0.5, n_estimators=500,
                      random_state=42)

Based on the evaluation metrics, the optimized cuML RandomForestRegressor model with Bayesian optimization shows significantly improved performance compared to the baseline scikit-learn RandomForestRegressor model. 
The optimized model achieves a near-perfect R² score of 0.999993, while the MAE and RMSE values are also very low, indicating minimal prediction errors.
The improvement can be attributed to both the GPU acceleration provided by cuML and the effective hyperparameter tuning through Bayesian optimization.
GPU acceleration allows for faster training and prediction times, while Bayesian optimization helps find the optimal hyperparameters for the specific dataset, leading to improved model accuracy.
Best hyperparameters: OrderedDict([('max_depth', 14), ('max_features', 0.9907794811380449), ('min_samples_leaf', 1), ('min_samples_split', 4), ('n_estimators', 100)])

In [55]:
# Train the Random Forest model on the training data with Bayesian optimized hyperparameters using Standard Scaler
# model = RandomForestRegressor(n_estimators=100, random_state=42, 
model = RandomForestRegressor(max_depth=14, max_features=0.9907794811380449, min_samples_leaf=1, min_samples_split=4, n_estimators=100) 
model.fit(X_train,y_train)

RandomForestRegressor(max_depth=14, max_features=0.9907794811380449,
                      min_samples_split=4)

In [ ]:
# Train the Random Forest model on the training data using MinMax Scaler
model = RandomForestRegressor(bootstrap=False, max_depth=None, max_features=0.5, min_samples_leaf=1, min_samples_split=2, n_estimators=500, random_state=42)
model.fit(X_train_minmax,y_train)

RandomForestRegressor(bootstrap=False, max_features=0.5, n_estimators=500,
                      random_state=42)

In [ ]:
# Train the Random Forest model on the training data using RobustScaler Scaler
model = RandomForestRegressor(bootstrap=False, max_depth=None, max_features=0.5, min_samples_leaf=1, min_samples_split=2, n_estimators=500, random_state=42)
model.fit(X_train_robust,y_train)

RandomForestRegressor(bootstrap=False, max_features=0.5, n_estimators=500,
                      random_state=42)

In [ ]:
# Train the Random Forest model on the training data using MaxAbsScaler Scaler
model = RandomForestRegressor(bootstrap=False, max_depth=None, max_features=0.5, min_samples_leaf=1, min_samples_split=2, n_estimators=500, random_state=42)
model.fit(X_train_maxabs,y_train)

RandomForestRegressor(bootstrap=False, max_features=0.5, n_estimators=500,
                      random_state=42)

In [ ]:
# Train the Random Forest model on the training data using Normalizer Scaler
model = RandomForestRegressor(bootstrap=False, max_depth=None, max_features=0.5, min_samples_leaf=1, min_samples_split=2, n_estimators=500, random_state=42)
model.fit(X_train_normalized,y_train)

RandomForestRegressor(bootstrap=False, max_features=0.5, n_estimators=500,
                      random_state=42)

In [ ]:
# Optimize the hyperparameters of the XGBoostRegressor model using 5-fold cross-validation with GridSearchCV. 

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Define the parameter grid for XGBoost
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Instantiate an XGBRegressor object with GPU support
xgb_model = XGBRegressor(tree_method='hist', device='cuda', random_state=42)

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_standard, y_train)

# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Train a new XGBoost model with the best hyperparameters
best_xgb_model = XGBRegressor(**grid_search.best_params_, tree_method='hist', device='cuda', random_state=42)
best_xgb_model.fit(X_train_standard, y_train)

# Make predictions on the test data
y_pred_best_xgb = best_xgb_model.predict(X_test_standard)

# Evaluate the optimized model
r2_best_xgb = r2_score(y_test, y_pred_best_xgb)
rmse_best_xgb = np.sqrt(mean_squared_error(y_test, y_pred_best_xgb))

print(f"Optimized XGBoost R2: {r2_best_xgb}")
print(f"Optimized XGBoost RMSE: {rmse_best_xgb}")

Best hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 150, 'subsample': 0.8}
Optimized XGBoost R2: 0.900440165864996
Optimized XGBoost RMSE: 0.005113935916892656

In [99]:
# Train the XGBoost model on the training data using Standard Scaler
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Instantiate an XGBRegressor object
model = XGBRegressor(colsample_bytree=0.8, learning_rate=0.2, max_depth=7, n_estimators=150, subsample=0.8, random_state=42)
# Best hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 150, 'subsample': 0.8}
# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
# y_pred_xgb = xgb_model.predict(X_test_standard)

# Evaluate the model's performance
# r2_xgb = r2_score(y_test, y_pred_xgb)
# rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

# print(f"XGBoost R2: {r2_xgb}")
# print(f"XGBoost RMSE: {rmse_xgb}")

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.2, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=150,
             n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
# Optimize the hyperparameters of the LightGBMRegressor model using 5-fold cross-validation with GridSearchCV.

from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

# Define the parameter grid for LightGBMRegressor
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'num_leaves': [31, 63, 127],
    'boosting_type': ['gbdt', 'dart'],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0, 0.1, 1],
}

# Instantiate a LightGBMRegressor object
lgbm_model = LGBMRegressor(device='cpu')

# Create a GridSearchCV object
grid_search = GridSearchCV(
    estimator=lgbm_model, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1
)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Train a new LightGBMRegressor model with the best hyperparameters
best_lgbm_model = LGBMRegressor(**grid_search.best_params_, device='cpu')
best_lgbm_model.fit(X_train, y_train)


Best hyperparameters: {'boosting_type': 'gbdt', 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 150, 'num_leaves': 63, 'reg_alpha': 0, 'reg_lambda': 1}


In [ ]:
# Train the LightGBMRegressor model on the training data using Standard Scaler
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Instantiate a LightGBMRegressor object
model = LGBMRegressor(boosting_type= 'gbdt', learning_rate=0.2, max_depth=7, n_estimators=150, num_leaves=63, reg_alpha=0, reg_lambda=1,device='cpu')

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
# y_pred_lgbm = model.predict(X_test)

# Evaluate the model's performance
# r2_lgbm = r2_score(y_test, y_pred_lgbm)
# rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))

# print(f"LightGBM R2: {r2_lgbm}")
# print(f"LightGBM RMSE: {rmse_lgbm}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.171839 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3126
[LightGBM] [Info] Number of data points in the train set: 6563, number of used features: 16
[LightGBM] [Info] Start training from score 1.000067
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

LGBMRegressor(device='cpu', learning_rate=0.2, max_depth=7, n_estimators=150,
              num_leaves=63, reg_alpha=0, reg_lambda=1)

In [ ]:
# Optimize the hyperparameters of the SVR model using GridSearchCV with 5-fold cross-validation.

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Define the parameter grid for SVR
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 0.5],
    'gamma': ['scale', 'auto', 0.1, 1],
}

# Instantiate an SVR object with RBF kernel
svr_model = SVR(kernel='rbf')

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=svr_model, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_scaled_standard, y_train)

# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Train a new SVR model with the best hyperparameters
best_svr_model = SVR(**grid_search.best_params_, kernel='rbf')
best_svr_model.fit(X_train_scaled_standard, y_train)

# Make predictions on the test data
y_pred_best_svr = best_svr_model.predict(X_test_scaled_standard)

# Evaluate the optimized model
r2_best_svr = r2_score(y_test, y_pred_best_svr)
rmse_best_svr = np.sqrt(mean_squared_error(y_test, y_pred_best_svr))

print(f"Optimized SVR R2: {r2_best_svr}")
print(f"Optimized SVR RMSE: {rmse_best_svr}")



Best hyperparameters: {'C': 10, 'epsilon': 0.01, 'gamma': 0.1}
Optimized SVR R2: 0.8139168782477013
Optimized SVR RMSE: 0.0069914409020128115


In [133]:
# Train an SVR model with RBF kernel using the training data with StandardScaler and evaluate its performance using R2 score and RMSE on the testing data.
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Instantiate an SVR object with RBF kernel
model = SVR(C=10, epsilon=0.01, gamma=0.1,kernel='rbf')

# Fit the SVR model to the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
# y_pred_svr = model.predict(X_test)

# Evaluate the model's performance
# r2_svr = r2_score(y_test, y_pred_svr)
# rmse_svr = np.sqrt(mean_squared_error(y_test, y_pred_svr))

# print(f"SVR R2: {r2_svr}")
# print(f"SVR RMSE: {rmse_svr}")

SVR(C=10, epsilon=0.01, gamma=0.1)

## Model Evaluation

<p align="justify">Now that we have trained our model, the next step is to evaluate its performance. For evaluation, we will use the R² score, a common metric for regression models that measures how well the model explains the variance in the response variable (UHI index). Scikit-learn provides many other metrics that can be used for evaluation, and you can also write custom code for more specific evaluation needs.</p>


### In-Sample Evaluation
<p align="justify">We will be evaluating our model's performance using the R² score on the training data. It is important to note that this is in-sample performance testing, which involves evaluating the model on the training dataset. These metrics are not truly indicative of the model's ability to generalize. You should reserve testing on the test data before drawing final conclusions about your model's performance.</p>


<p align="justify">In this section, we make predictions on the training set and store them in the <b><i>insample_predictions</i></b> variable. The R² score is then calculated to gauge the model's performance on the training data. It is important to keep in mind that this evaluation is for the training set, and further testing on the test set is necessary to assess the model's generalizability.</p>


In [56]:
# Make predictions on the training data
insample_predictions = model.predict(X_train)

In [57]:
# calculate R-squared score for in-sample predictions
Y_train = y_train.tolist()
r2_score(Y_train, insample_predictions)

0.9502561736856473

### Out-Sample Evaluation

When evaluating a machine learning model, it is essential to correctly and fairly evaluate the model's ability to generalize. This is because models have a tendency to overfit the dataset they are trained on. To estimate the out-of-sample performance, we will predict on the test data now. 

In [58]:
# Make predictions on the test data
outsample_predictions = model.predict(X_test)

In [59]:
# calculate R-squared score for out-sample predictions
Y_test = y_test.tolist()
r2_score(Y_test, outsample_predictions)

0.8581182211402271

## Submission

Once you are satisfied with your model, you can proceed to make a submission. To do this, you will need to use your model to predict the Urban Heat Island (UHI) index for a set of test coordinates provided in the <b>"Submission_template.csv"</b> file and upload the results to the challenge platform.

In [28]:
#Reading the coordinates for the submission
test_file = pd.read_csv('Submission_template.csv')
test_file.head()

,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,NaN
1,-73.971928,40.788875,NaN
2,-73.967080,40.789080,NaN
3,-73.972550,40.789082,NaN
4,-73.969697,40.787953,NaN


In [29]:
def map_ground_data_buffer(ground_csv, submission_csv, buffer_m=200):
    """
    Maps ground features from the Integrated_UHI_Dataset_200m.csv onto the submission points by creating 
    a buffer (in meters) around each submission location and aggregating the ground data values within the buffer.
    
    Parameters:
      ground_csv: Path to the ground dataset CSV file.
                  (Assumed to have columns: 'Longitude', 'Latitude', 'building_d', 'impervious', 
                  'avg_temp', 'avg_wind_speed', 'avg_solar_flux')
      submission_csv: Path to the submission file (with columns: 'Longitude', 'Latitude')
      buffer_m: Buffer radius in meters (e.g., 50, 100, 150, 200)
      
    Returns:
      A DataFrame that contains the submission coordinates along with the aggregated ground features.
    """
    # Read the ground data and submission data
    df_ground = pd.read_csv(ground_csv)
    df_sub = pd.read_csv(submission_csv)
    
    # Create GeoDataFrames using coordinate columns (assumed to be 'Longitude' and 'Latitude')
    gdf_ground = gpd.GeoDataFrame(
        df_ground,
        geometry=gpd.points_from_xy(df_ground.Longitude, df_ground.Latitude),
        crs="EPSG:4326"
    )
    gdf_sub = gpd.GeoDataFrame(
        df_sub,
        geometry=gpd.points_from_xy(df_sub.Longitude, df_sub.Latitude),
        crs="EPSG:4326"
    )
    
    # For accurate distance buffering in meters, reproject to a metric CRS (e.g., EPSG:3857)
    gdf_ground = gdf_ground.to_crs(epsg=3857)
    gdf_sub = gdf_sub.to_crs(epsg=3857)
    
    # Create a buffer for each submission point (buffer_m in meters)
    gdf_sub["buffer"] = gdf_sub.geometry.buffer(buffer_m)
    
    # List to store aggregated features for each submission record.
    aggregated_features = []
    
    # Loop over each submission point and find ground data points within the buffer
    for idx, sub_row in gdf_sub.iterrows():
        buff_geom = sub_row["buffer"]
        # Select all ground points within the buffer geometry
        points_within = gdf_ground[gdf_ground.geometry.within(buff_geom)]
        if not points_within.empty:
            # Aggregate the ground features (here we take the mean)
            agg = {
                "building_d": points_within["building_d"].mean(),
                "impervious": points_within["impervious"].mean(),
                "avg_temp": points_within["avg_temp"].mean(),
                "avg_wind_speed": points_within["avg_wind_speed"].mean(),
                "avg_solar_flux": points_within["avg_solar_flux"].mean()
            }
        else:
            agg = {
                "building_d": None,
                "impervious": None,
                "avg_temp": None,
                "avg_wind_speed": None,
                "avg_solar_flux": None
            }
        aggregated_features.append(agg)
    
    # Convert aggregated features to a DataFrame
    df_agg = pd.DataFrame(aggregated_features)
    
    # Combine (merge) with the submission data based on row order
    df_merged = pd.concat([gdf_sub.reset_index(drop=True), df_agg], axis=1)
    # Drop geometry columns if not needed
    df_merged = df_merged.drop(columns=["geometry", "buffer"])
    
    return df_merged
'''
# Example usage:
if __name__ == "__main__":
    merged_ground_df = map_ground_data_buffer("Integrated_UHI_Dataset_200m.csv", 
                                                "Submission_template.csv", 
                                                buffer_m=200)
    print("Sample of merged ground features:")
    print(merged_ground_df.head())
    # Optionally, save for later use.
    merged_ground_df.to_csv("submission_ground_features_mapped.csv", index=False)
'''

'\n# Example usage:\nif __name__ == "__main__":\n    merged_ground_df = map_ground_data_buffer("Integrated_UHI_Dataset_200m.csv", \n                                                "Submission_template.csv", \n                                                buffer_m=200)\n    print("Sample of merged ground features:")\n    print(merged_ground_df.head())\n    # Optionally, save for later use.\n    merged_ground_df.to_csv("submission_ground_features_mapped.csv", index=False)\n'

In [30]:
# Mapping ground data for submission.
val_data_ground = map_ground_data_buffer("Integrated_UHI_Dataset_200m.csv", "Submission_template.csv", buffer_m=200)

In [31]:
# Mapping satellite data for submission.
# val_data = map_satellite_data('S2_AllBands.tiff', 'Submission_template.csv')
val_data_satellite = map_satellite_data_buffer("S2_AllBands.tiff", "Submission_template.csv", buffer_m=200)


Mapping values with 200 m buffer:   0%|          | 0/1040 [00:00<?, ?it/s]

Mapping values with 200 m buffer: 100%|██████████| 1040/1040 [00:39<00:00, 26.65it/s]


In [32]:
# Combining val_data_ground and val_data_satellite into a single dataset.
val_data = combine_two_datasets(val_data_ground,val_data_satellite)
val_data.head()

,Longitude,Latitude,UHI Index,building_d,impervious,avg_temp,avg_wind_speed,avg_solar_flux,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12
0,-73.971665,40.788763,NaN,0.000071,3.197655,27.3,3.8,349.0,1116.587745,1226.046996,1360.754313,1414.881618,1656.922665,2173.941106,2374.327186,2354.713266,2445.699584,2286.048780,1907.582391
1,-73.971928,40.788875,NaN,0.000077,3.038272,27.3,3.8,349.0,1108.466899,1204.832753,1338.692799,1398.419861,1637.300232,2137.396632,2333.681185,2308.498839,2402.080139,2257.235192,1889.696864
2,-73.967080,40.789080,NaN,0.000044,4.225734,27.3,3.8,349.0,852.127305,942.464604,1068.649613,1070.921475,1321.616895,2072.905413,2362.734682,2337.282570,2450.966092,1957.826889,1517.947650
3,-73.972550,40.789082,NaN,0.000076,3.047948,27.3,3.8,349.0,1111.176681,1193.264723,1320.558596,1383.697204,1607.414634,2073.831053,2263.631767,2231.424152,2310.889946,2174.771565,1830.134444
4,-73.969697,40.787953,NaN,0.000051,4.692358,27.3,3.8,349.0,1120.465794,1244.011898,1371.138013,1411.389649,1656.049970,2217.123736,2438.536585,2397.465794,2498.408685,2312.516359,1958.831053


In [33]:
# Calculate NDVI (Normalized Difference Vegetation Index) and handle division by zero by replacing infinities with NaN.
val_data['NDVI'] = (val_data['B08'] - val_data['B04']) / (val_data['B08'] + val_data['B04'])
val_data['NDVI'] = val_data['NDVI'].replace([np.inf, -np.inf], np.nan)  # Replace infinities with NaN

# Calculate NDBI (Normalized Difference Built-up Index) and handle division by zero by replacing infinities with NaN.
val_data['NDBI'] = (val_data['B11'] - val_data['B08']) / (val_data['B11'] + val_data['B08'])
val_data['NDBI'] = val_data['NDBI'].replace([np.inf, -np.inf], np.nan)  # Replace infinities with NaN

# Calculate NDWI (Normalized Difference Water Index) and handle division by zero by replacing infinities with NaN.
val_data['NDWI'] = (val_data['B03'] - val_data['B08']) / (val_data['B03'] + val_data['B08'])
val_data['NDWI'] = val_data['NDWI'].replace([np.inf, -np.inf], np.nan)  # Replace infinities with NaN

# Calculate NDMI (Normalized Difference Moisture Index) and handle division by zero by replacing infinities with NaN.
val_data['NDMI'] = (val_data['B08'] - val_data['B11']) / (val_data['B08'] + val_data['B11'])
val_data['NDMI'] = val_data['NDMI'].replace([np.inf, -np.inf], np.nan)  # Replace infinities with NaN

In [34]:
val_data.head()

,Longitude,Latitude,UHI Index,building_d,impervious,avg_temp,avg_wind_speed,avg_solar_flux,B01,B02,...,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,NDMI
0,-73.971665,40.788763,NaN,0.000071,3.197655,27.3,3.8,349.0,1116.587745,1226.046996,...,2173.941106,2374.327186,2354.713266,2445.699584,2286.048780,1907.582391,0.249319,-0.014796,-0.267519,0.014796
1,-73.971928,40.788875,NaN,0.000077,3.038272,27.3,3.8,349.0,1108.466899,1204.832753,...,2137.396632,2333.681185,2308.498839,2402.080139,2257.235192,1889.696864,0.245508,-0.011228,-0.265905,0.011228
2,-73.967080,40.789080,NaN,0.000044,4.225734,27.3,3.8,349.0,852.127305,942.464604,...,2072.905413,2362.734682,2337.282570,2450.966092,1957.826889,1517.947650,0.371563,-0.088346,-0.372477,0.088346
3,-73.972550,40.789082,NaN,0.000076,3.047948,27.3,3.8,349.0,1111.176681,1193.264723,...,2073.831053,2263.631767,2231.424152,2310.889946,2174.771565,1830.134444,0.234495,-0.012857,-0.256439,0.012857
4,-73.969697,40.787953,NaN,0.000051,4.692358,27.3,3.8,349.0,1120.465794,1244.011898,...,2217.123736,2438.536585,2397.465794,2498.408685,2312.516359,1958.831053,0.258890,-0.018036,-0.272336,0.018036


In [60]:
# Extracting specific columns (B01,B02,B05,B06,B07,B08A,B12,NDVI,NDBI,NDMI and NDWI) from the validation dataset
submission_val_data=val_data.loc[:,['B01','B02','B05','B06','B07','B8A','B12','NDVI','NDBI','NDMI','NDWI', 'building_d', 'impervious', 'avg_temp', 'avg_wind_speed', 'avg_solar_flux']]
submission_val_data.head()

,B01,B02,B05,B06,B07,B8A,B12,NDVI,NDBI,NDMI,NDWI,building_d,impervious,avg_temp,avg_wind_speed,avg_solar_flux
0,1116.587745,1226.046996,1656.922665,2173.941106,2374.327186,2445.699584,1907.582391,0.249319,-0.014796,0.014796,-0.267519,0.000071,3.197655,27.3,3.8,349.0
1,1108.466899,1204.832753,1637.300232,2137.396632,2333.681185,2402.080139,1889.696864,0.245508,-0.011228,0.011228,-0.265905,0.000077,3.038272,27.3,3.8,349.0
2,852.127305,942.464604,1321.616895,2072.905413,2362.734682,2450.966092,1517.947650,0.371563,-0.088346,0.088346,-0.372477,0.000044,4.225734,27.3,3.8,349.0
3,1111.176681,1193.264723,1607.414634,2073.831053,2263.631767,2310.889946,1830.134444,0.234495,-0.012857,0.012857,-0.256439,0.000076,3.047948,27.3,3.8,349.0
4,1120.465794,1244.011898,1656.049970,2217.123736,2438.536585,2498.408685,1958.831053,0.258890,-0.018036,0.018036,-0.272336,0.000051,4.692358,27.3,3.8,349.0


In [46]:
# Extracting specific columns (B01,B02,B05,B06,B07,B08A,B12,NDVI,NDBI,NDMI and NDWI) from the validation dataset
submission_val_data=val_data.loc[:,selected_features_boruta]
submission_val_data.head()

,B01,B06,NDVI,building_d,impervious,avg_temp,avg_wind_speed,avg_solar_flux
0,1116.587745,2173.941106,0.249319,0.000071,3.197655,27.3,3.8,349.0
1,1108.466899,2137.396632,0.245508,0.000077,3.038272,27.3,3.8,349.0
2,852.127305,2072.905413,0.371563,0.000044,4.225734,27.3,3.8,349.0
3,1111.176681,2073.831053,0.234495,0.000076,3.047948,27.3,3.8,349.0
4,1120.465794,2217.123736,0.258890,0.000051,4.692358,27.3,3.8,349.0


In [61]:
# Feature Scaling 
submission_val_data = submission_val_data.values
transformed_submission_data = sc.transform(submission_val_data)

In [62]:
#Making predictions
final_predictions = model.predict(transformed_submission_data)
final_prediction_series = pd.Series(final_predictions)

In [63]:
#Combining the results into dataframe
submission_df = pd.DataFrame({'Longitude':test_file['Longitude'].values, 'Latitude':test_file['Latitude'].values, 'UHI Index':final_prediction_series.values})

In [64]:
#Displaying the sample submission dataframe
submission_df.head()

,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,0.963863
1,-73.971928,40.788875,0.965328
2,-73.967080,40.789080,0.969393
3,-73.972550,40.789082,0.967272
4,-73.969697,40.787953,0.960213


In [65]:
#Dumping the predictions into a csv file.
submission_df.to_csv("submission_allbands_groundfeatures_200m_RF_BO.csv",index = False)

### Upload submission file on platform

Upload the submission.csv on the <a href ="https://challenge.ey.com">platform</a> to get score generated on scoreboard.

## Conclusion

<div align ="justify">Now that you have learned a basic approach to model training, it’s time to try your own approach! Feel free to modify any of the functions presented in this notebook. We look forward to seeing your version of the model and the results. Best of luck with the challenge!</div>